<a href="https://colab.research.google.com/github/komo135/forex-prediction/blob/master/ipynb/rl/rainbow(no_noisy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

In [4]:
%cd /content/drive/My Drive
from env import Env

try:
  import imp
  imp.reload(rainbow)
except:
  import rainbow

env = Env(step_size=96, available_assets_rate=1)
agent = rainbow.Agent(restore=not True, n=3, lr=1e-3, env=env)

/content/drive/My Drive


In [0]:
%cd /content

agent.run(train=True)

action probability: buy=0.3854166666666667, sell=0.3229166666666667, hold=0.2916666666666667
epoch: 5, total assets:4200.395839352066, growth_rate:0.04200395839352066

action probability: buy=0.6145833333333334, sell=0.052083333333333336, hold=0.3333333333333333
epoch: 10, total assets:500.6952191539952, growth_rate:0.005006952191539952

action probability: buy=0.375, sell=0.375, hold=0.25
epoch: 15, total assets:558.7794901068971, growth_rate:0.005587794901068971

action probability: buy=0.375, sell=0.2708333333333333, hold=0.3541666666666667
epoch: 20, total assets:491.939250779266, growth_rate:0.00491939250779266

action probability: buy=0.3541666666666667, sell=0.5208333333333334, hold=0.125
epoch: 25, total assets:59598.95458001546, growth_rate:0.5959895458001546

action probability: buy=0.3541666666666667, sell=0.4166666666666667, hold=0.22916666666666666
epoch: 30, total assets:449.41216783209273, growth_rate:0.004494121678320927

action probability: buy=0.625, sell=0.1770833333

In [0]:
import numpy as np
tree_idx, replay = agent.memory.sample(128)

states = np.array([a[0][0] for a in replay], np.float32)
new_states = np.array([a[0][3] for a in replay], np.float32)
actions = np.array([a[0][1] for a in replay]).reshape((-1, 1))
rewards = np.array([a[0][2] for a in replay], np.float32).reshape((-1, 1))

q = agent.model(states)
target_q = agent.target_model(new_states).numpy()
arg_q = np.sum(agent.model(new_states).numpy(), -1).reshape((-1,3))
arg_q = np.argmax(arg_q, -1)
# arg_q = [np.argmax(i) if 0.05 < np.random.rand() else np.random.randint(2) for i in arg_q]

q_backup = q.numpy()

for i in range(q.shape[0]):
  q_backup[i, actions[i]] = rewards[i] + agent.gamma * target_q[i, arg_q[i]]


error = tf.abs(q_backup - q)
# q_error = tf.maximum(self.tau * error, (self.tau - 1) * error)
loss = tf.where(error < 2, error ** 2 * .5, 2 * error - 0.5 * 2 ** 2)
loss *= tf.abs(agent.tau - tf.stop_gradient(tf.cast((q_backup - q) < 0, tf.float32)))
# loss = agent.huber_loss(q_backup, q)
# loss = tf.abs(agent.tau - tf.stop_gradient(tf.cast((q_backup - q) < 0, tf.float32))) * loss
# loss = tf.reduce_sum(loss, 2)

# print(q)

In [7]:
np.sum(agent.model.predict_on_batch(new_states), -1).astype(np.int32)

array([[ -8738,  -2308,   3198],
       [ -7927,  -2093,   2903],
       [ -7938,  -2096,   2904],
       [ -8143,  -2151,   2975],
       [ -7191,  -1898,   2633],
       [ -7957,  -2101,   2913],
       [ -8040,  -2123,   2945],
       [-11597,  -3066,   4244],
       [-11546,  -3050,   4234],
       [-10649,  -2814,   3895],
       [ -6851,  -1809,   2506],
       [ -8005,  -2114,   2926],
       [ -8044,  -2124,   2942],
       [ -8105,  -2141,   2965],
       [ -8263,  -2182,   3025],
       [ -8478,  -2240,   3097],
       [-19308,  -5104,   7072],
       [ -8052,  -2126,   2949],
       [ -8382,  -2214,   3063],
       [ -7699,  -2033,   2813],
       [ -8065,  -2130,   2948],
       [ -6871,  -1815,   2511],
       [-11797,  -3118,   4315],
       [-12348,  -3262,   4525],
       [ -7644,  -2020,   2790],
       [ -7768,  -2052,   2840],
       [ -6685,  -1765,   2442],
       [ -8347,  -2204,   3054],
       [ -8601,  -2273,   3142],
       [ -8624,  -2278,   3155],
       [ -

In [15]:
q[0]

<tf.Tensor: shape=(3, 200), dtype=float32, numpy=
array([[-1.44341821e+03, -1.42630444e+03, -1.39350195e+03,
        -1.34819775e+03, -1.33212659e+03, -1.32610681e+03,
        -1.27248438e+03, -1.25612744e+03, -1.19686047e+03,
        -1.15399597e+03, -1.13093823e+03, -1.11477649e+03,
        -1.09229602e+03, -1.07379114e+03, -1.02154083e+03,
        -9.86540710e+02, -9.53670837e+02, -9.38668213e+02,
        -9.25329285e+02, -9.11844971e+02, -8.92148865e+02,
        -8.64507324e+02, -8.46400208e+02, -8.13078064e+02,
        -7.94027527e+02, -7.75918335e+02, -7.44536743e+02,
        -7.21649902e+02, -7.03519653e+02, -6.81461914e+02,
        -6.68654541e+02, -6.65886047e+02, -6.38207825e+02,
        -6.24272217e+02, -6.20020142e+02, -6.09596436e+02,
        -6.04166931e+02, -6.00741882e+02, -5.87439087e+02,
        -5.68684448e+02, -5.59589111e+02, -5.42568909e+02,
        -5.32188965e+02, -5.24674805e+02, -5.16256104e+02,
        -4.98227325e+02, -4.74224426e+02, -4.55383728e+02,
      